<a href="https://colab.research.google.com/github/babatundeibukun/fastai_deep_learning/blob/main/training_mini_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#i need that data that contaons just 3's and 7's
#that is my first step
#importing all i need to make my life easy


#hide
from fastai.vision.all import *
#from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [82]:
path = untar_data(URLs.MNIST_SAMPLE)


In [83]:
#We can see what's in this directory by using ls, a method added by fastai
path.ls()

(#3) [Path('/root/.fastai/data/mnist_sample/valid'),Path('/root/.fastai/data/mnist_sample/labels.csv'),Path('/root/.fastai/data/mnist_sample/train')]

In [84]:
(path/'train').ls()


(#2) [Path('/root/.fastai/data/mnist_sample/train/7'),Path('/root/.fastai/data/mnist_sample/train/3')]

In [85]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
threes

(#6131) [Path('/root/.fastai/data/mnist_sample/train/3/10.png'),Path('/root/.fastai/data/mnist_sample/train/3/10000.png'),Path('/root/.fastai/data/mnist_sample/train/3/10011.png'),Path('/root/.fastai/data/mnist_sample/train/3/10031.png'),Path('/root/.fastai/data/mnist_sample/train/3/10034.png'),Path('/root/.fastai/data/mnist_sample/train/3/10042.png'),Path('/root/.fastai/data/mnist_sample/train/3/10052.png'),Path('/root/.fastai/data/mnist_sample/train/3/1007.png'),Path('/root/.fastai/data/mnist_sample/train/3/10074.png'),Path('/root/.fastai/data/mnist_sample/train/3/10091.png'),Path('/root/.fastai/data/mnist_sample/train/3/10093.png'),Path('/root/.fastai/data/mnist_sample/train/3/10097.png'),Path('/root/.fastai/data/mnist_sample/train/3/10099.png'),Path('/root/.fastai/data/mnist_sample/train/3/10116.png'),Path('/root/.fastai/data/mnist_sample/train/3/10125.png'),Path('/root/.fastai/data/mnist_sample/train/3/10137.png'),Path('/root/.fastai/data/mnist_sample/train/3/10141.png'),Path('/ro

In [86]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors),len(seven_tensors)

(6131, 6265)

#My MUSINGS

In [87]:
#I need the validation dataset but we will talk about it later
#this problem won't exist for real if i had a datset but now , i don't ,
#there is no point bitching around , I have to solve my problem.


#i am interested in the current shape of my seven_tensors , i tried it but
#realized that  list dont have shapes , so what do i do?
#convert the list into into rank3 tensor


In [88]:
stacked_seven = torch.stack(seven_tensors).float()/255
stacked_seven.shape

torch.Size([6265, 28, 28])

In [89]:
stacked_three = torch.stack(three_tensors).float()/255
stacked_three.shape

torch.Size([6131, 28, 28])

#MY THOUGHTS

In [90]:
#great , I crossed that first hurdle , but that isnt surely enough .
#I need to find a way to concatenate these two into one one long list

#MY MUSING

In [91]:
#the code below stressed as i had too much error in my thinking, i knew from my normal pandas work that i can concatenate only when i have them in a list like format
#but i hesistated , this hesitation is scary because how can I?  Is this not common knowledge?
# lastly i fumbled when i was doing torch.cat , initially , i dint put cat, , at some point i even put tensor!! damn it!

In [92]:
train_x = torch.cat([stacked_seven, stacked_three]).view(-1, 28*28)

In [93]:
# i am interested to seeing what this looks like
#i expect a rank-2 tensor
train_x.shape

torch.Size([12396, 784])

#MY MUSING

In [94]:
#but we need a train_y , because whats deep learning without a truth value.

train_y = tensor(len(seven_tensors)*[0]  + len(three_tensors)*[1]).unsqueeze(1)

In [95]:
len(train_y)

12396

In [96]:
#but in pytorch , when we index a dataset , we want it to return a a tuple that looks like this , (a,b)
#so :
ds = list(zip(train_x, train_y))
x, y = ds[0]
x.shape, y

(torch.Size([784]), tensor([0]))

In [97]:

valid_3_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape,valid_7_tens.shape


(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [98]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [168]:
valid_dl = DataLoader(valid_dset, batch_size=256)


In [99]:
#now we have a great working dataset , now lets go to the first step.

#STEP 1 : Initialize the weights

In [130]:
def init_params(size, std=1.0):
  return (torch.randn(size)*std).requires_grad_()

#STEP 2 : Make predictions

but to be sure we are doing well on predictions. I want to first check the prediction for the first dataset


In [131]:
#let me initialize my weights

weights = init_params(784)
bias = init_params(1)

In [132]:
y_preds = ((train_x[0])*(weights.T)).sum() + bias
y_preds

tensor([-8.0561], grad_fn=<AddBackward0>)

Now lets do it on all the dataset , I mean , we can loop but that would be stupid for sure when there is matrix multiplication.

In [133]:
def linear1(xb):
  return xb@weights + bias

In [134]:
#the code is literally batch gradient descent

In [135]:
preds = linear1(train_x)
preds

tensor([ -8.0561,   4.5531, -16.6335,  ...,   7.9521,   1.5664,  -5.6556],
       grad_fn=<AddBackward0>)

# Current Accuracy

In [136]:
correct = (preds > 0.0).float() == train_y
correct

tensor([[ True, False,  True,  ..., False, False,  True],
        [ True, False,  True,  ..., False, False,  True],
        [ True, False,  True,  ..., False, False,  True],
        ...,
        [False,  True, False,  ...,  True,  True, False],
        [False,  True, False,  ...,  True,  True, False],
        [False,  True, False,  ...,  True,  True, False]])

In [137]:
correct.float().mean()

tensor(0.5013)

#step  3 : The loss function for a binary clasification

In [138]:
def mnist_loss(preds, trgts):
  preds = preds.sigmoid()
  return torch.where(trgts==1, 1-preds, preds).mean()

In [139]:
#the goal of the BCE is to keep the loss as low as possible

lets test what we have above

In [140]:
trgts = tensor([1.0, 0, 1.0])
preds = tensor([0.9, 0.4, 0.8])

In [141]:
torch.where(trgts==1, 1-preds, preds)

tensor([0.1000, 0.4000, 0.2000])

In [142]:
torch.where(trgts==1, 1-preds, preds).mean()

tensor(0.2333)

In [143]:
mnist_loss(preds, trgts)

tensor(0.3993)

#putting it all together

In [144]:
#lets call out dataLoader
df = DataLoader(ds, batch_size= 256, shuffle= True)
xb,yb = first(df)
xb.shape,yb.shape


(torch.Size([256, 784]), torch.Size([256, 1]))

#Testing for batchsize of 4 and seeing the possibilities.

In [145]:
#step 1
#find the y_preds

preds = linear1(train_x[:4])
preds

tensor([ -8.0561,   4.5531, -16.6335,  -3.2143], grad_fn=<AddBackward0>)

In [146]:
#step 2
#find the loss
loss = mnist_loss(preds, train_y[:4])
loss

tensor(0.2571, grad_fn=<MeanBackward0>)

In [147]:
#step 3
#calculate grad
loss.backward()

In [148]:
#check for the gradients calculated
#since it is too large , I will deal with the mean
weights.grad.mean(), bias.grad

(tensor(0.0014), tensor([0.0119]))

#calculating the gradient

In [149]:
#I want to put all of this into a function to make it so compact.

def calc_grad(xb, yb, model):
  preds = model(xb)  #i struggled here, ultimatly , model is anything that takes in an  argument.
  loss = mnist_loss(preds, yb)
  loss.backward()


#Train the weight for an epoch.

In [177]:
def train_epoch(model, lr, params):
  for xb, yb in df:
    calc_grad(xb, yb, model)
    for p in params:
      p.data -= p.grad * lr
      p.grad.zero_()

#lets calculate the accuracy for the 4 batch size

In [178]:
correct = (preds > 0.5).float() == train_y[:4]
correct

tensor([[ True, False,  True,  True],
        [ True, False,  True,  True],
        [ True, False,  True,  True],
        [ True, False,  True,  True]])

In [179]:
correct.float().mean()

tensor(0.7500)

#Now i want to create a function for batch accuracy

In [180]:
def batch_accuracy(model, yb):
  correct = (preds > 0.5).float()== yb
  return correct.float().mean()


In [181]:
#let me test the above code and see if it is working
batch_accuracy(linear1(train_x[:4]), train_y[:4])

tensor(0.7500)

#validation accuracy

In [182]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)


#Great that was a random result since the weight initialized was random

In [183]:
#let me test the code i just wrote
validate_epoch(linear1)

0.5034

#Train for an epoch

In [184]:
lr =1.0
params = weights,bias

In [186]:
train_epoch(linear1, lr, params)

In [187]:
validate_epoch(linear1)

0.5034

In [174]:
for i in range(20):
    train_epoch(linear1, lr, params)
    print(validate_epoch(linear1), end=' ')

0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 0.5034 